In [77]:
import requests
import pandas as pd

API_KEY = 'R9UPFmY5MUYOj3vq1gBg62698NdO0SzL94jlOWBa'
BASE_URL = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'

# Define parameters for API request with more fields added incrementally
params = {
    'api_key': API_KEY,
    'fields': 'id,school.name,school.city,school.state,latest.admissions.admission_rate.overall,latest.cost.tuition.in_state,latest.cost.tuition.out_of_state,latest.student.size,latest.admissions.sat_scores.average.overall,latest.admissions.act_scores.average.cumulative,latest.earnings.10_yrs_after_entry.median,latest.completion.rate_suppressed.overall,latest.aid.pell_grant_rate,latest.student.demographics.gender.men,latest.student.demographics.gender.women',
    'per_page': 100,
    'page': 0
}

all_data = []

# Loop through multiple pages
for page in range(0, 50):  
    params['page'] = page
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        data = response.json()
        results = data['results']
        all_data.extend(results) 
    else:
        print(f"Error on page {page}: {response.status_code}")
        break


df = pd.DataFrame(all_data)
print(f"Total rows fetched: {len(df)}")
df.to_csv('college_data.csv', index=False)
df.to_csv('college_data_new.csv', index=False)


Total rows fetched: 5000


In [78]:
import pandas as pd
import requests
df = pd.read_csv('/Users/adityakumar/US.College_Analysis/college_data_new.csv')
df.sample(1)

,latest.admissions.admission_rate.overall,latest.cost.tuition.in_state,latest.cost.tuition.out_of_state,latest.student.size,latest.admissions.sat_scores.average.overall,latest.earnings.10_yrs_after_entry.median,latest.completion.rate_suppressed.overall,latest.aid.pell_grant_rate,school.name,school.city,school.state,id
3576,0.9821,5818,5818,290,NaN,24908,0.383,0.7812,Pontifical Catholic University of Puerto Rico-...,Arecibo,PR,241395


In [79]:
# Rename specific columns using .rename()
df = df.rename(columns={
    'school.name': 'College_Name',
    'school.city': 'City',
    'school.state': 'State',
    'latest.admissions.admission_rate.overall': 'Admission_Rate',
    'latest.cost.tuition.in_state': 'In-State_Tuition',
    'latest.cost.tuition.out_of_state': 'Out-of-State_Tuition',
    'latest.student.size': 'Student_Size',
    'latest.admissions.sat_scores.average.overall': 'Average_SAT_Score',
    'latest.earnings.10_yrs_after_entry.median': 'Median_Earnings_10_Years_After',
    'latest.completion.rate_suppressed.overall': 'Completion_Rate',
    'latest.aid.pell_grant_rate': 'Pell_Grant_Rate',
})

df.to_csv('college_data.csv', index=False)


In [80]:
import ydata_profiling as yp
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title ="US College")
profile.to_file("US_College.html")

from IPython.display import FileLink
FileLink("US_College.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 67.22it/s]


/Users/adityakumar/US.College_Analysis/US_College.html

Dropping NAN rows

In [81]:
new_df = df.dropna()
new_df.shape
df = new_df

In [82]:
df.drop(columns=['City', 'State', 'id'], inplace= True)

/var/folders/tf/nb2206b133550_hpm_b_xr_40000gn/T/ipykernel_30278/824456240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['City', 'State', 'id'], inplace= True)


In [83]:
import ydata_profiling as yp
from ydata_profiling import ProfileReport

profile = ProfileReport(new_df, title ="US College")
profile.to_file("US_College_01.html")

from IPython.display import FileLink
FileLink("US_College_01.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 137.63it/s]


/Users/adityakumar/US.College_Analysis/US_College_01.html

outlier detection and treatment

In [84]:
#skewed cols are  ['Admission_Rate', 'Student_Size', 'Average_SAT_Score', 'Median_Earnings_10_Years_After', 'Pell_Grant_Rate']

cols_to_cap = ['Admission_Rate', 'Average_SAT_Score', 'Median_Earnings_10_Years_After', 'Pell_Grant_Rate']

def cap_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Calculate the number of outliers
    num_outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    total_values = len(df[col])
    outlier_percentage = (num_outliers / total_values) * 100
    print(f"Percentage of outliers in {col}: {outlier_percentage:.2f}%")

    #Capping
    df[col] = df[col].apply(lambda x: upper_bound if x > upper_bound else lower_bound if x < lower_bound else x)

def trim_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Calculate the number of outliers
    num_outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    total_values = len(df[col])
    outlier_percentage = (num_outliers / total_values) * 100
    print(f"Percentage of outliers in {col}: {outlier_percentage:.2f}%")

    #Trimming
    trimmed_df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return trimmed_df

for col in cols_to_cap:
    cap_outliers(df, col)

df_trimmed = trim_outliers(df, 'Student_Size')


print(f"New DataFrame shape after trimming Student_Size: {df_trimmed.shape}")


Percentage of outliers in Admission_Rate: 5.83%
Percentage of outliers in Average_SAT_Score: 3.01%
Percentage of outliers in Median_Earnings_10_Years_After: 5.27%
Percentage of outliers in Pell_Grant_Rate: 2.35%
Percentage of outliers in Student_Size: 11.01%
New DataFrame shape after trimming Student_Size: (946, 9)


In [85]:
#normally distributed are ['In-State_Tuition','Out-of-State_Tuition','Completion_Rate']
from scipy.stats import zscore

cols_to_check = ['In-State_Tuition','Out-of-State_Tuition','Completion_Rate']

def outliers_detection(df, col, threshold=3):
    z_scores = zscore(df[col])

    # Calculate the number of outliers
    num_outliers = ((z_scores > threshold) | (z_scores < -threshold)).sum()
    total_values = len(df[col])
    outlier_percentage = (num_outliers / total_values) * 100
    print(f"Percentage of outliers in {col}: {outlier_percentage:.2f}%")

for col in cols_to_check:
    outliers_detection(df, col)
    

Percentage of outliers in In-State_Tuition: 0.00%
Percentage of outliers in Out-of-State_Tuition: 0.00%
Percentage of outliers in Completion_Rate: 0.00%


Transformation

In [86]:
columns_skewness = ['Average_SAT_Score','Pell_Grant_Rate','Admission_Rate', 'Student_Size', 'Median_Earnings_10_Years_After']

for col in columns_skewness:
    skewness = df[col].skew()
    print(f"Skewness of {col}: {skewness:.4f}")


Skewness of Average_SAT_Score: 0.5952
Skewness of Pell_Grant_Rate: 0.5488
Skewness of Admission_Rate: -0.9663
Skewness of Student_Size: 2.8794
Skewness of Median_Earnings_10_Years_After: 0.5639


Moderate Skewed: 0.5 - 1.0 -> Applying BoxCox

In [87]:
from scipy.stats import boxcox
import numpy as np

columns_to_transform = ['Average_SAT_Score','Pell_Grant_Rate', 'Admission_Rate','Median_Earnings_10_Years_After']

lambdas = {}

for col in columns_to_transform:
    if col == 'Pell_Grant_Rate' or col == 'Median_Earnings_10_Years_After':
        df[col] = df[col] + 0.0001
    
    # Ensure there are no negative values for Box-Cox
    if any(df[col] <= 0):
        df[col] = df[col] + abs(df[col].min()) + 1
    
    df[col], fitted_lambda = boxcox(df[col])
    lambdas[col] = fitted_lambda 
    
    print(f"Applied Box-Cox transformation on {col} with lambda: {fitted_lambda:.4f}")

Applied Box-Cox transformation on Average_SAT_Score with lambda: -0.9632
Applied Box-Cox transformation on Pell_Grant_Rate with lambda: 0.6378
Applied Box-Cox transformation on Admission_Rate with lambda: 2.0743
Applied Box-Cox transformation on Median_Earnings_10_Years_After with lambda: 0.2646


Student Size Highly positive skewed -> Applying Reciprocal Transformation 

In [88]:
df['Student_Size'] = np.log1p(df['Student_Size'])


In [89]:
columns_skewness = ['Average_SAT_Score','Pell_Grant_Rate','Admission_Rate', 'Student_Size', 'Median_Earnings_10_Years_After']

for col in columns_skewness:
    skewness = df[col].skew()
    print(f"Skewness of {col}: {skewness:.4f}")

Skewness of Average_SAT_Score: 0.0088
Skewness of Pell_Grant_Rate: 0.0267
Skewness of Admission_Rate: -0.3311
Skewness of Student_Size: 0.1779
Skewness of Median_Earnings_10_Years_After: 0.0101


In [90]:
df.shape

(1063, 9)

Normalizing

In [91]:
from sklearn.preprocessing import MinMaxScaler
cols_to_normalise = ['Admission_Rate', 'In-State_Tuition','Out-of-State_Tuition', 'Student_Size'
                , 'Average_SAT_Score', 'Median_Earnings_10_Years_After','Completion_Rate', 'Pell_Grant_Rate']

scaler = MinMaxScaler()
df[cols_to_normalise] = scaler.fit_transform(df[cols_to_normalise])


In [92]:
import ydata_profiling as yp
from ydata_profiling import ProfileReport

profile = ProfileReport(df, title ="US College After Feature Engineering")
profile.to_file("US_College_FE.html")

from IPython.display import FileLink
FileLink("US_College_FE.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 113.74it/s]


/Users/adityakumar/US.College_Analysis/US_College_FE.html

In [93]:
df.sample(1)

,Admission_Rate,In-State_Tuition,Out-of-State_Tuition,Student_Size,Average_SAT_Score,Median_Earnings_10_Years_After,Completion_Rate,Pell_Grant_Rate,College_Name
1113,0.945546,0.71952,0.71952,0.471203,0.539645,0.602546,0.670638,0.512755,Wartburg College


Weights based on user features

In [94]:
weights = {
    'Admission_Rate': 0.1,
    'In-State_Tuition': 0.2,
    'Out-of-State_Tuition': 0.1,
    'Student_Size': 0.1,
    'Average_SAT_Score': 0.2,
    'Median_Earnings_10_Years_After': 0.2,
    'Completion_Rate': 0.1,
    'Pell_Grant_Rate': 0.05
}

In [95]:
df['Ranking_Score'] = (
    df['Admission_Rate'] * weights['Admission_Rate'] +
    df['In-State_Tuition'] * weights['In-State_Tuition'] +
    df['Out-of-State_Tuition'] * weights['Out-of-State_Tuition'] +
    df['Student_Size'] * weights['Student_Size'] +
    df['Average_SAT_Score'] * weights['Average_SAT_Score'] +
    df['Median_Earnings_10_Years_After'] * weights['Median_Earnings_10_Years_After'] +
    df['Completion_Rate'] * weights['Completion_Rate'] +
    df['Pell_Grant_Rate'] * weights['Pell_Grant_Rate']
)

In [96]:
X = df.drop(columns = ['College_Name', 'Ranking_Score']).values
y = df['Ranking_Score'].values
College_Name = df['College_Name']

In [97]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, random_state= 42)

# Extract X and y for train/test sets
X_train = train_data[['Admission_Rate', 'In-State_Tuition', 'Out-of-State_Tuition', 'Student_Size', 
                      'Average_SAT_Score', 'Median_Earnings_10_Years_After', 'Completion_Rate', 'Pell_Grant_Rate']].values
y_train = train_data['Ranking_Score'].values

X_test = test_data[['Admission_Rate', 'In-State_Tuition', 'Out-of-State_Tuition', 'Student_Size', 
                    'Average_SAT_Score', 'Median_Earnings_10_Years_After', 'Completion_Rate', 'Pell_Grant_Rate']].values
y_test = test_data['Ranking_Score'].values

results= []

XGBOOST

In [98]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
import numpy as np

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    'objective': 'rank:pairwise',  
    'eval_metric': 'ndcg',         
    'learning_rate': 0.1,
    'max_depth': 5,
}

rank_model_xgb = xgb.train(params, dtrain, num_boost_round=100)

y_pred_xgb = rank_model_xgb.predict(dtest)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)


results.append({
    'Model': 'XGBoost',
    'MSE': mse_xgb,
    'MAE': mae_xgb,
    'R2_Score': r2_xgb
})


GradientBoosting

In [99]:
from sklearn.ensemble import GradientBoostingRegressor

rank_model_gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
rank_model_gb.fit(X_train, y_train)
y_pred_gb = rank_model_gb.predict(X_test)

mse_gb = mean_squared_error(y_test, y_pred_gb)
mae_gb = mean_absolute_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)


results.append({
    'Model': 'Gradient Boosting',
    'MSE': mse_gb,
    'MAE': mae_gb,
    'R2_Score': r2_gb
})

results_df = pd.DataFrame(results)

Random Forest

In [100]:
from sklearn.ensemble import RandomForestRegressor
rank_model_rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

rank_model_rf.fit(X_train, y_train)
y_pred_rf = rank_model_rf.predict(X_test)

mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

# Store the results
results.append({
    'Model': 'Random Forest',
    'MSE': mse_rf,
    'MAE': mae_rf,
    'R2_Score': r2_rf
})

In [101]:
results_df = pd.DataFrame(results)
results_df

,Model,MSE,MAE,R2_Score
0,XGBoost,17.537856,4.113250,-1001.401889
1,Gradient Boosting,0.000235,0.010672,0.986550
2,Random Forest,0.000712,0.020346,0.959278


5 CV

In [102]:
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
r2_scorer = make_scorer(r2_score)
cv_r2_gb = cross_val_score(rank_model_gb, X_train, y_train, cv=kf, scoring=r2_scorer)

print(f"Gradient Boosting 5CV R2 scores: {cv_r2_gb}")
print(f"Gradient Boosting 5CV Average R2: {np.mean(cv_r2_gb)}")

Gradient Boosting 5CV R2 scores: [0.9837734  0.98045765 0.96996104 0.9827855  0.97999979]
Gradient Boosting 5CV Average R2: 0.9793954737722432


Converting back to college name

In [109]:
college_names = test_data['College_Name'].values

results = pd.DataFrame({
    'College_Name': college_names,
    'Predicted_Score': y_pred_gb
})
top_colleges = results.sort_values(by='Predicted_Score', ascending=False)


In [110]:
print(top_colleges)

                              College_Name  Predicted_Score
106                  University of Chicago         0.866695
50                         Yale University         0.861538
191                Northeastern University         0.852272
114        Case Western Reserve University         0.844761
45   Massachusetts Institute of Technology         0.835870
..                                     ...              ...
148        Elizabeth City State University         0.327745
15                      Randall University         0.322640
44                 Alcorn State University         0.298701
85     Mississippi Valley State University         0.278681
65                  Wilberforce University         0.277762

[213 rows x 2 columns]


In [111]:
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(rank_model_gb, f)

In [112]:
df.to_csv('college_data_new.csv', index=False)